# CHAPTER 2 - The Tools of the Trade in Quantum Computing - PennyLane Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.



In [ ]:
pip install pennylane==0.26

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install pennylane-qiskit==0.24.0

In [ ]:
import pennylane as qml

In [ ]:
dev = qml.device('default.qubit', wires = 2)

In [ ]:
def qc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()

In [5]:
qcirc = qml.QNode(qc, dev) # Assemble the circuit & the device.
qcirc() # Run it!

array([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])

In [6]:
@qml.qnode(dev) # We add this decorator to use the device dev.
def qcirc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()
    
# Now qcirc is already a QNode. We can just run it!
qcirc()

array([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,  0.        +0.j])

In [7]:
dev = qml.device('default.qubit', wires = 1)
@qml.qnode(dev)
def qcirc(theta):
    qml.RX(theta, wires = 0)
    return qml.state()

In [8]:
print(qml.draw(qcirc)(theta = 2))

0: ──RX(2.00)─┤  State


In [9]:
dev = qml.device('default.qubit', wires = 3)

# Get probabilities
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [1, 2]) # Only the last 2 wires.
prob = qcirc()
print("Probs. wires [1, 2] with H in wire 1:", prob)

# Get a sample, not having specified shots in the device.
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.sample(wires = 0) # Only the first wire.
s1 = qcirc(shots = 4) # We specify the shots here.
print("Sample 1 after H:", s1)

# Get a sample with shots in the device.
dev = qml.device('default.qubit', wires = 2, shots = 4)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.sample() # Will sample all wires.
s2 = qcirc()
print("Sample 2 after H x I:", s2)

Probs. wires [1, 2] with H in wire 1: [0.5 0.  0.5 0. ]
Sample 1 after H: [[0]
 [0]
 [1]
 [1]]
Sample 2 after H x I: [[0 0]
 [1 0]
 [1 0]
 [0 0]]


/home/furtado/anaconda3/envs/qmlPraticalOptMaisAtual/lib/python3.11/site-packages/pennylane/workflow/qnode.py:808: PennyLaneDeprecationWarning: Specifying 'shots' when executing a QNode is deprecated and will be removed in v0.44. Please set shots on QNode initialization, or use qml.set_shots instead.
  shots = self._get_shots(kwargs)
/home/furtado/anaconda3/envs/qmlPraticalOptMaisAtual/lib/python3.11/site-packages/pennylane/devices/device_api.py:201: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


In [10]:
dev = qml.device('qiskit.aer', wires = 2)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
s = qcirc()
print("The probabilities are", s)

The probabilities are [0.48828125 0.51171875]


/home/furtado/anaconda3/envs/qmlPraticalOptMaisAtual/lib/python3.11/site-packages/pennylane_qiskit/qiskit_device_legacy.py:144: UserWarning: The analytic calculation of expectations, variances and probabilities is only supported on statevector backends, not on the aer_simulator. Such statistics obtained from this device are estimates based on samples.
  warnings.warn(self.analytic_warning_message.format(self.backend_name), UserWarning)


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [11]:
ibm_token = "1234"

In [12]:
# Import the modules.
from qiskit import *
from qiskit_ibm_runtime import *
#from qiskit.providers.ibmq import *

# Save our token if we haven't already.
#IBMQ.save_account(ibm_token)

QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token=ibm_token, set_as_default=True, overwrite=True) 
# QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token) 
# Load saved credentials
service = QiskitRuntimeService()

# Load the account and get the name of the least busy backend.
#prov = IBMQ.load_account()
#bck = service.backends(simulator = False)
bck = service.least_busy(operational=True, simulator=False, min_num_qubits=5)
# Invoke the PennyLane IBMQ device.
dev = qml.device('qiskit.remote', wires = 1,
    backend = bck)

# Send a circuit and get some results!
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
print(qcirc())

qiskit_runtime_service.__init__:WARNING:2026-02-02 15:43:53,288: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: Testes. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-02-02 15:43:53,915: Loading instance: Testes, plan: open
qiskit_runtime_service.backends:WARNING:2026-02-02 15:43:55,700: Using instance: Testes, plan: open
/home/furtado/anaconda3/envs/qmlPraticalOptMaisAtual/lib/python3.11/site-packages/pennylane_qiskit/qiskit_device.py:62: UserWarning: Expected an integer number of shots, but received shots=None. A default number of shots will be selected by the Qiskit backend. The analytic calculation of results is not supported on this devi

RuntimeInvalidStateError: 'Unable to retrieve result for job d60f1dabju6s73bcu3qg. Job was cancelled.'